In [2]:
import os
import requests
import json
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

In [3]:
# Begin by getting a list of all artist urls
MAX_ARTISTS = 6215
top_path = "https://tzvetnik.online/articles/artist/"

artist_urls = [top_path+str(i+1) for i in range(MAX_ARTISTS)]


In [5]:
# Step two: get all the show urls
top_path = "https://tzvetnik.online"
show_urls = []
for url in tqdm(artist_urls):
    response = requests.get(url)

    try:
        response.raise_for_status() # Raises an exception for HTTP errors
        artist_html = response.text
        soup = BeautifulSoup(artist_html, 'html.parser')

        article_previews = soup.find('div', class_="article-previews")
        anchor_tags = article_previews.find_all('a')
           
        for anchor_tag in anchor_tags:
            href_value = top_path + anchor_tag.get('href')
            show_urls.append(href_value)       

    except:
        print(f"error requesting {url} | status code: {response.status_code}")


show_urls = list(set(show_urls))
with open("show_urls.txt", "w") as file:
    for item in show_urls:
        file.write(item+"\n")
len(show_urls)

  0%|          | 0/6215 [00:00<?, ?it/s]

error requesting https://tzvetnik.online/articles/artist/26 | status code: 200
error requesting https://tzvetnik.online/articles/artist/151 | status code: 200
error requesting https://tzvetnik.online/articles/artist/164 | status code: 200
error requesting https://tzvetnik.online/articles/artist/180 | status code: 200
error requesting https://tzvetnik.online/articles/artist/231 | status code: 200
error requesting https://tzvetnik.online/articles/artist/235 | status code: 200
error requesting https://tzvetnik.online/articles/artist/258 | status code: 200
error requesting https://tzvetnik.online/articles/artist/280 | status code: 200
error requesting https://tzvetnik.online/articles/artist/446 | status code: 200
error requesting https://tzvetnik.online/articles/artist/457 | status code: 200
error requesting https://tzvetnik.online/articles/artist/505 | status code: 200
error requesting https://tzvetnik.online/articles/artist/544 | status code: 200
error requesting https://tzvetnik.online/

2917

In [13]:
with open("./show_urls.txt", "w") as file:
    for item in show_urls:
        file.write(item+"\n")
len(show_urls[:10])

10

In [16]:
data = []
for url in tqdm(show_urls[:10]):
    dir = os.path.basename(url)
    show = {"dir" : dir}
    imgs = []
    response = requests.get(url)

    try:
        response.raise_for_status()
        show_html = response.text
        soup = BeautifulSoup(show_html, "html.parser")
        action_texts = soup.find_all("action-text-attachment")
        for action_text in action_texts:
            try:
                img_tag = action_text.find("img")
                img_url = img_tag.get("src")
                caption_tag = action_text.find("figcaption")
                caption = caption_tag.get_text(strip=True)
                imgs.append((img_url, caption))
            except:
                pass        
        
        if len(imgs) != 0:
            show["imgs"] = imgs
            data.append(show)
        
    except:
        print(f"error with url: {url} | response code: {response.status_code}")

with open("data.json", "w") as json_file:
    json.dump(data, json_file)





  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
len(data)

10

In [18]:
data[0]["dir"]

'the-days-are-just-packed-off-site-group-show-at-the-pool-heybeliada-istanbul'

In [ ]:
# TODO: This section must be fixed so that the data is downloaded in the proper
# ImageFolder format, see finetuning0.ipynb for details and helpful links.

from concurrent.futures import ThreadPoolExecutor

# Function to download a single image
def download_image(img_url, img_path):
    try:
        response = requests.get(img_url, stream=True)
        response.raise_for_status()

        # Check if the image file already exists, skip if it does
        if not os.path.exists(img_path):
            with open(img_path, 'wb') as img_file:
                for chunk in response.iter_content(chunk_size=8192):
                    img_file.write(chunk)
    except Exception as e:
        print(f"Failed to download {img_url}: {e}")

# Create the 'img' directory if it doesn't exist
img_directory = 'img'
os.makedirs(img_directory, exist_ok=True)

# Loop through the list of dictionaries
#for item in tqdm(data):
    dir_name = item['dir']
    img_urls = item['imgs']

    # Create a subdirectory within 'img' based on 'dir' value
    subdirectory_path = os.path.join(img_directory, dir_name)
    os.makedirs(subdirectory_path, exist_ok=True)

    # Create a ThreadPoolExecutor with a maximum of 5 threads (adjust as needed)
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []

        # Download and save images in parallel
        for img_url in tqdm(img_urls):
            img_filename = os.path.basename(img_url[0])
            img_path = os.path.join(subdirectory_path, img_filename)

            futures.append(executor.submit(download_image, img_url[0], img_path))

        # Wait for all download tasks to complete
        for future in futures:
            future.result()
